# Poor hint Migration 
This notebook demonstrates how an LLM performs when presented with a Spring Security migration issue without an example.

Installing pre requisites and configuring Kai with `GPT-3.5-turbo`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [2]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 
# Initialize the model provider using GPT-4o via OpenAI
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "granite-8b-code-instruct-128k",
        "base_url": "https://granite-8b-code-instruct-maas-apicast-production.apps.prod.rhoai.rh-aiservices-bu.com:443/v1",
        "api_key": os.getenv("OPENAI_API_KEY"),},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



/Users/sraghuna/local_dev/kai-notebook-solution-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [3]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [4]:
analysis_rule=""""\
    
- ruleID: spring-security-replace-authenticationmanagerbuilder-00001
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  category: mandatory
  effort: 2
  labels:
    - konveyor.io/source=spring-boot2
    - konveyor.io/target=spring-boot3+
  message: |
    Overriding `configure(AuthenticationManagerBuilder)` is **discouraged** in Spring Security 5.7.
    Instead, define a `UserDetailsService` and `PasswordEncoder` bean.
    ```
  when:
    java.referenced:
      location: METHOD
      pattern: configure(org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder) *
    """

Similar example showing before and after migration

In [5]:
example_before_code="""\

@Configuration
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        auth.inMemoryAuthentication()
            .withUser(user);
    }
}

"""

example_after_code="""\
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
"""

In [6]:
prompt = f"""

The following java code needs to be migrated from springboot2 to springboot3.

```java
{before_code}
```
I will provide you with an example before/after code migration
```java
{example_before_code}
{example_after_code}
```
The issue has been identified by an analysis rule

{analysis_rule}

Can you provide a refactored version of this code that is compatible with springboot3?
Also, create an hint based on your suggestions.

"""

In [7]:

response = await rendered_llm_call(prompt)

Refactored code:

```java
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
```

Hint:

```java
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
```

Explanation:

The refactored code is a direct replacement for the original code, but with the updated UserDetailsService and PasswordEncoder beans. The UserDetailsService bean is created using the User.withDefaultPasswordEncoder() method to define the user with a username, password, and roles. The PasswordEncoder bean is created using the new BCryptPasswordEncoder() method to encode the password.

The hint provides a complete solution for the code migration, including the refactored code and the UserDetailsService and PasswordEncoder beans.


### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  Yes           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           |  Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  No           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | Yes        |